# Ray Crash Course - Tasks

© 2019-2020, Anyscale. All Rights Reserved

![Anyscale Academy](../images/AnyscaleAcademy_Logo_clearbanner_141x100.png)

Let's quickly explore the Ray API using some examples that demonstrate how Ray enables horizontal scalability.

> **Tip:** For more about Ray, see [ray.io](https://ray.io) or the [Ray documentation](https://docs.ray.io/en/latest/).

Evaluate the next _three_ notebook cells (shift+return, if you're new to notebooks), down to and including `run_simulations(...)`, and watch what happens:

In [ ]:
from task_lesson_util import make_dmaps, run_simulations, stop_simulations
from pi_calc import str_large_n

In [ ]:
Ns = [1000, 10000, 100000]

dmaps = make_dmaps(Ns)

dmaps[0] + dmaps[1] + dmaps[2]

In [ ]:
run_simulations(dmaps)

In [ ]:
# TIP: If you want to stop them, uncomment and run the next line:
# stop_simulations(dmaps)

(If you can't see it, click [here](../images/Pi-estimates.png) for a screen grab.)

What we just did was estimate $\pi$ (~3.14159) using a [_Monte Carlo_ technique](https://en.wikipedia.org/wiki/Monte_Carlo_method), where we randomly sampled a _uniform distribution_, one with equal probably of picking any point in a square.

It works like this. Imagine each blue square is a piece of paper 2 meters by 2 meters you put on a wall. The circle inside each one has radius 1 meter.

Now suppose you throw $N$ darts at each paper. We're seeing $N = {\sim}1000, {\sim}10000, {\sim}100000$ examples. (This will be hard on your wall, so don't try this at home...)

Some darts will land inside the circle, call them $n$, and the rest will land outside, $N-n$. The area of a circle is ${\pi}r^{2}$ and the area of a square is $(2r)^{2} = 4r^{2}$. The ratio of $n/N$ _approximately_ equals the ratio of the circle area over the square area, ${\pi}r^{2}/4r^{2} = {\pi}/4$. (Does it make sense that this ratio is independent of the actual radius value?).

In other words,

$\pi/4 \approx n/N$

$\pi \approx 4n/N$

So, to approximate $\pi$, we can count the number of darts thrown and the number that land inside the circle.

You probably noticed three things while the simulations were running or after they finished:

1. The accuracy improved for larger $N$... well usually. Sometimes a lower $N$ simulation gets "lucky" and does as well as a higher $N$. In a real experiment, we would do many runs and then compute the average and standard deviation. (We'll do that below.)
2. Because each $N$ is 10 times the $N$ to the left, it took roughly 10 times as long for the second to finish compared to the first, etc.
3. The updates in the second and third simulations appeared to go faster as the neighbors to the left finished.

What this means is that if we really want a good estimate of $\pi$, we have to do runs with large $N$, but then we wait longer. Ideally, to get fast _and_ accurate results, we would do as much work as possible in parallel, leveraging all the CPU cores available on our machine ... or our cluster. 

Let's use [_Ray_](https://ray.io) to achieve this.

> **Note:** There is a lot of Python code used in this notebook, both for calculating Pi and for the graphs. We won't show most of it, but all the code can be found in this directory, `pi_calc.py` (calculating $\pi$) and `task_lesson_util.py` (graphics). 

## Parallelism with Ray

We did the previous calculation without fully exploiting all available cores. In a cluster, the rest of the cores _on the rest of the machines_ would be idle, too.

We can use Ray to parallelize a lot of this work. Let's see how.

In [ ]:
import numpy as np
import math, statistics, time
import ray

Some constants we'll use (and explain) below.

In [ ]:
num_workers = 8
trials = 20  

### Let's Start Ray

If you are running these tutorials on your laptop, each notebook will start a mini-cluster when we call `ray.init()` below. They we'll shut it down at the end of the notebook, so be sure to evaluate the last cell in each notebook that calls `ray.shutdown()`.

If you are learning on the Anyscale platform, your environment is already running a Ray cluster with one or more nodes. So, when we call `ray.init()`, it will connect to that running cluster. You should still run the `ray.shutdown()` cell at the end of each notebook, but it won't shutdown the whole cluster.

For more details on the options you can pass to `ray.init()`, see lesson 6, [Exploring Ray API Calls](06-Exploring-Ray-API-Calls.ipynb). 
If you are interested in the details of running a Ray cluster using the `ray` CLI, see lesson 7, [Running Ray Clusters](07-Running-Ray-Clusters.ipynb) and see also the corresponding [Ray documentation](https://docs.ray.io/en/latest/starting-ray.html#using-ray-on-a-cluster). There is also a script `tools/start-ray.sh` that can you play with. (It was used in an earlier version of these tutorials.)

The `ignore_reinit_error=True` argument tells Ray not to complain if we rerun the cell; Ray will just ignore the request to initialize.

In [ ]:
ray.init(ignore_reinit_error=True)

> **Tip:** Having trouble starting Ray? See the [Troubleshooting](../reference/Troubleshooting-Tips-Tricks.ipynb) tips.

Although we don't need it immediately, we'll use the Ray Dashboard to watch performance metrics like CPU utilization.

The next cell prints the URL for the Ray Dashboard. **This is only correct if you are running this tutorial on a laptop.** Click the link to open the dashboard.

If you are running on the Anyscale platform, use the URL provided by your instructor to open the Dashboard.

In [ ]:
print(f'Dashboard URL: http://{ray.get_webui_url()}')

Let's define a function to do the Pi calculation that simplifies the code we used above for graphing purposes. We won't do the "dart graphs" from now on, because they add a lot of overhead that would obscure the performance

This function estimates $\pi$ for the number of samples requested. It uses [NumPy](https://docs.scipy.org/doc/numpy/index.html). If you're not familiar with it, the implementation details aren't essential to understand, but the comments try to explain them.

In [ ]:
def estimate_pi(num_samples):
    xs = np.random.uniform(low=-1.0, high=1.0, size=num_samples)   # Generate num_samples random samples for the x coordinate.
    ys = np.random.uniform(low=-1.0, high=1.0, size=num_samples)   # Generate num_samples random samples for the y coordinate.
    xys = np.stack((xs, ys), axis=-1)                              # Like Python's "zip(a,b)"; creates np.array([(x1,y1), (x2,y2), ...]).
    inside = xs*xs + ys*ys <= 1.0                                  # Creates a predicate over all the array elements.
    xys_inside = xys[inside]                                       # Selects only those "zipped" array elements inside the circle.
    in_circle = xys_inside.shape[0]                                # Return the number of elements inside the circle.
    approx_pi = 4.0*in_circle/num_samples                          # The Pi estimate.
    return approx_pi

Let's try it:

In [ ]:
Ns = [10000, 50000, 100000, 500000, 1000000] #, 5000000, 10000000]  # Larger values take a long time on small VMs and machines!
maxN = Ns[-1]
maxN

In [ ]:
fmt = '{:10.5f} seconds: pi ~ {:7.6f}, stddev = {:5.4f}, error = {:5.4f}%'

In [ ]:
def try_it(n, trials):
    print('trials = {:3d}, N = {:s}: '.format(trials, str_large_n(n, padding=12)), end='')   # str_large_n imported above.
    start = time.time()
    pis = [estimate_pi(n) for _ in range(trials)]
    approx_pi = statistics.mean(pis)
    stdev = statistics.stdev(pis)
    duration = time.time() - start
    error = (100.0*abs(approx_pi-np.pi)/np.pi)
    print(fmt.format(duration, approx_pi, stdev, error))   # str_large_n imported above.
    return trials, n, duration, approx_pi, stdev, error

The next cell will take many seconds to complete (depending on your setup). As soon as it starts, switch to the Ray Dashboard you opened above. Notice the total CPU and memory utilizations at the top while the cell runs. 

> **Tip:** If all the following trials finish in under a few seconds for the largest `n` value in `Ns` and the largest number of trials, consider changing `Ns` above to add larger values.

In [ ]:
data_ns = [try_it(n, trials) for n in Ns]

In [ ]:
data_trials = [try_it(maxN, trials) for trials in range(5,20,2)]

(We'll graph the data below.)

The CPU utilization never gets close to 100%. On a four-core machine, for example, the number will be about 25%. (The Ray process meters will stay at or near zero until later in this notebook.)

So, this runs on one core, while the other cores are idle. Now we'll try with Ray.

### From Python Functions to Ray Tasks

You create a Ray _task_ by decorating a normal Python function with `@ray.remote`. These tasks will be scheduled across your Ray cluster (or your laptop CPU cores).

> **Tip:** The [Ray Package Reference](https://ray.readthedocs.io/en/latest/package-ref.html) in the [Ray Docs](https://ray.readthedocs.io/en/latest/) is useful for exploring the API features we'll learn.

Here is a Ray task for `estimate_pi`. All we need is a wrapper around the original function.

In [ ]:
@ray.remote
def ray_estimate_pi(num_samples):
    return estimate_pi(num_samples)

Let's try it. To invoke a task, you use `function.remote(args)`:

In [ ]:
ray_estimate_pi.remote(100)

What is this `ObjectRef`? A Ray task is an asynchronous computation. The `ObjectRef` returned is a _future_ that we use to retrieve the resulting value from the task when it completes. We use `ray.get(ref)` to get it:

In [ ]:
ref = ray_estimate_pi.remote(100)
print(ray.get(ref))

We can also work with lists of refs:

In [ ]:
refs = [ray_estimate_pi.remote(n) for n in [100, 1000, 10000]]
print(ray.get(refs))

Okay, let's try our test run again with our Ray task. We'll need a new "try it" function, because of the different task invocation logic. This function doesn't need to be a Ray task, however, so no `@ray.remote` decorator is required.

In [ ]:
def ray_try_it(n, trials):
    print('trials = {:3d}, N = {:s}: '.format(trials, str_large_n(n, padding=12)), end='')   # str_large_n imported above.
    start = time.time()
    refs = [ray_estimate_pi.remote(n) for _ in range(trials)]
    pis = ray.get(refs)
    approx_pi = statistics.mean(pis)
    stdev = statistics.stdev(pis)
    duration = time.time() - start
    error = (100.0*abs(approx_pi-np.pi)/np.pi)
    print(fmt.format(duration, approx_pi, stdev, error))   # str_large_n imported above.
    return trials, n, duration, approx_pi, stdev, error

In [ ]:
ray_data_ns = [ray_try_it(n, trials) for n in Ns]

In [ ]:
ray_data_trials = [ray_try_it(maxN, trials) for trials in range(5,20,2)]

The durations should be shorter than the non-Ray numbers. Let's graph our results and see. It will be easier if we first convert the `*data_*` lists to NumPy arrays, so they are easier to slice.

In [ ]:
np_data_ns         = np.array(data_ns)
np_data_trials     = np.array(data_trials)
np_ray_data_ns     = np.array(ray_data_ns)
np_ray_data_trials = np.array(ray_data_trials)

In [ ]:
from bokeh_util import two_lines_plot, means_stddevs_plot  # Some plotting utilities in `./bokeh_util.py`.
from bokeh.plotting import show, figure
from bokeh.layouts import gridplot

First a linear plot of the results

In [ ]:
two_lines = two_lines_plot(
    "N vs. Execution Times (Smaller Is Better)", 'N', 'Time', 'No Ray', 'Ray', 
    np_data_ns[:,1], np_data_ns[:,2], np_ray_data_ns[:,1], np_ray_data_ns[:,2],
    x_axis_type='linear', y_axis_type='linear')
show(two_lines, plot_width=800, plot_height=400)

(If you can't see it, click [here](../images/Pi-Ns-vs-times-linear.png). Note that this image is from a run that included the larger values for `N` that are commented out in the definition of `Ns` above.)

For relatively small `N` values, the performance overhead of overhead of Ray is a larger percentage of the calculation, so the overall performance benefit is less. However, as `N` increases, the advantage of Ray increases. Both plots are roughly-linear, because we are CPU bound, but Ray's execution time/N is lower. On a full cluster, the times could be dramatically better for larger `N`. 

A log-log plot shows the lower-N behavior more clearly:

In [ ]:
two_lines = two_lines_plot(
    "N vs. Execution Times (Smaller Is Better)", 'N', 'Time', 'No Ray', 'Ray', 
    np_data_ns[:,1], np_data_ns[:,2], np_ray_data_ns[:,1], np_ray_data_ns[:,2])
show(two_lines, plot_width=800, plot_height=400)

(If you can't see it, click [here](../images/Pi-Ns-vs-times.png).)

What about execution times as a function of the number of trials, for a fixed `N`?

In [ ]:
two_lines = two_lines_plot(
    "Trials (N=10,000,000) vs. Execution Times (Smaller Is Better)", 'Trials', 'Time', 'No Ray', 'Ray', 
    np_data_trials[:,0], np_data_trials[:,2], np_ray_data_trials[:,0], np_ray_data_trials[:,2], 
    x_axis_type='linear', y_axis_type='linear')
show(two_lines, plot_width=800, plot_height=400)

(If you can't see it, click [here](../images/Pi-trials-vs-times.png).)


Let's plot the approximate mean values and the standard deviations over the `num_workers` trials for each `N`.

In [ ]:
pi_without_ray_plot = means_stddevs_plot(
  np_data_ns[:,1], np_data_ns[:,3], np_data_ns[:,4], title = 'π Results without Ray')
# Use a grid to make it layout better.
pi_without_ray_grid = gridplot([[pi_without_ray_plot]], plot_width=1000, plot_height=400)
show(pi_without_ray_grid)

(If you can't see it, click [here](../images/Pi-Results-without-Ray.png).)
You may have to use the "crossed-arrows" controls scroll horizontally (click and drag) to see all of the graph.

As you might expect, for low `N` values, the error bars are large and the mean estimate is poor, but for higher `N`, the errors grow smaller and results converge to the correct value.

With Ray, the plot will look similar, because we did the same calculation, just faster:

In [ ]:
pi_with_ray_plot = means_stddevs_plot(
  np_ray_data_ns[:,1], np_ray_data_ns[:,3], np_ray_data_ns[:,4], title = 'π Results with Ray')
# Use a grid to make it layout better.
pi_with_ray_grid = gridplot([[pi_with_ray_plot]], plot_width=1000, plot_height=400)
show(pi_with_ray_grid)

(If you can't see it, click [here](../images/Pi-Results-with-Ray.png).)

## ray.get() vs. ray.wait()

Calling `ray.get(ids)` blocks until all the tasks have completed that correspond to the input `ids`. That has been fine for this tutorial so far, but what if you're waiting for a number of tasks, where some will finish more quickly than others? What if you would like to process the completed results as they become available, even while other tasks are still running? That's where `ray.wait()` is recommended. Here we'll provide a brief example. For more details, see the Advanced Ray, [Ray Tasks Revisited](../advanced-ray/01-Ray-Tasks-Revisited.ipynb) lesson.

In [ ]:
@ray.remote
def ray_estimate_pi2(n, trial):
    time.sleep(trial)
    return n, trial, estimate_pi(n)

In [ ]:
def ray_try_it2(ns, trials):
    start = time.time()
    refs = [ray_estimate_pi2.remote(n, trial) for trial in trials for n in ns]
    still_running = list(refs)
    while len(still_running) > 0:
        finished, still_running = ray.wait(still_running)
        ns_trials_pis = ray.get(finished)   # won't block
        print(f'{ns_trials_pis}, elapsed time = {time.time() - start} secs')

Observe what happens next:

In [ ]:
ray_try_it2([100000,1000000,1000000], [2,4,6])

## Exercises

Try one or more of the following exercises to practice improving scalable performance using Ray. In particular, think about the granularity of tasks where Ray is most beneficial.

See the [solutions notebook](solutions/Ray-Crash-Course-Solutions.ipynb) for a discussion of solutions to these exercises.

### Exercise 1

As currently written, the memory footprint of `estimate_pi` scales linearly with `N`, because it allocates two NumPy arrays of size `N`. This limits the size of `N` we can evaluate (as I confirmed by locking up my laptop...). However, this isn't actually necessary. We could do the same calculation in "blocks, for example `m` blocks of size `N/m` and then combine the results. Furthermore, there's no dependencies between the calculations with those blocks, giving us further potential speed-up by parellelizing them with Ray.

Adapt `ray_estimate_pi` to use this technique. Pick some `N` value above which the calculation is done in blocks. Compare the performance of the old vs. new implementation. 

As you do this exercise, you might ponder the fact that we often averaged multiple trials for a given `N` and then ask yourself, what's the difference between averaging `10` trials for `N = 1000` vs. `1` trial for `N = 10000`, for example?

### Exercise 2

What `N` value is needed to get a reliable estimate to five decimal places, `3.1415` (for some definition of "reliable")? If you have a powerful machine or a cluster, you could try a higher accuracy. You'll need to use the solution to Exercise 1 or you can make a guess based on the results we've already seen in this notebook.

### Exercise 3

For small computation problems, Ray adds enough overhead that its benefits are outweighed. You can see from the performance graphs above that smaller `N` or smaller trial values will likely cause the curves to cross. Try small values of `N` and small trial numbers. When do the lines cross? Try timing individual runs for small `N` around the crossing point. What can you infer from this "tipping point" about appropriate sizing of tasks, at least for your test environment?

Run the next cell when you are finished with this notebook:

In [ ]:
ray.shutdown()  # "Undo ray.init()". Terminate all the processes started in this notebook.

The next lesson, [Ray Actors](02-Ray-Actors.ipynb), introduces Ray's tool for distributed computation _with state_, **actors**, which builds on the familiar Python concept of classes.               